In [1]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('Flair is pretty neat!')
classifier.predict(sentence)
# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

2019-09-10 12:12:33,945 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/classy-imdb-en-rnn-cuda%3A0/imdb-v0.4.pt not found in cache, downloading to /scratch/tmpucxf2h3h


100%|██████████| 1501979561/1501979561 [00:25<00:00, 58597264.71B/s]

2019-09-10 12:12:59,881 copying /scratch/tmpucxf2h3h to cache at /homes/stirunag/.flair/models/imdb-v0.4.pt


2019-09-10 12:13:02,995 removing temp file /scratch/tmpucxf2h3h
2019-09-10 12:13:03,236 loading file /homes/stirunag/.flair/models/imdb-v0.4.pt
Sentence above is:  [POSITIVE (0.6636102199554443)]


In [1]:
import pandas as pd
import numpy as np
import itertools
import codecs
import utils
import os

In [14]:
# get the current working directory
data_path = os.getcwd()+'/Datasets/'
# os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['CLASS'] = 'drug_gene'

doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['CLASS'] = 'gene_disease'
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t', na_filter = False)
EUADR_drug_disease['CLASS'] = 'drug_disease'


EUADR_temp = EUADR_drug_target.append(EUADR_target_disease).append(EUADR_drug_disease)

EUADR_temp_1 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'PA']
EUADR_temp_2 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'NA']
EUADR_temp = EUADR_temp_1.append(EUADR_temp_2)
EUADR_temp.rename(columns={"SENTENCE": "text", "ASSOCIATION_TYPE": "label"}, inplace=True)

EUADR_temp.reset_index(inplace=True)

In [15]:
# Get GAD dataset

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_Y_N'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_Y_N = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_Y_N['CLASS'] = 'gene_disease'

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_F'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_F = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_F['CLASS'] = 'gene_disease'

GAD_temp = GAD_target_disease_Y_N



GAD_temp['GAD_ASSOC'] = GAD_temp['GAD_ASSOC'].apply(lambda x: x.replace('Y', 'PA').replace('N', 'NA').replace('F', 'FA'))

GAD_temp.rename(columns={"GAD_ASSOC": "label", "GAD_CONCLUSION": "text"}, inplace=True)
GAD_temp.reset_index(inplace=True)

In [16]:
df = EUADR_temp[['label','text']].append(GAD_temp[['label','text']])
df.reset_index(inplace=True)

In [17]:
df.drop("index", axis=1, inplace=True)


In [3]:
data_path = os.getcwd()+'/Datasets/'
data = pd.read_csv(data_path+'CoMAGC/'+'CoMAGC.csv')

In [4]:
data_loc = '/home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/CoMAGC/'

data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv(data_loc+'train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv(data_loc+'test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv(data_loc+'dev.csv', sep='\t', index = False, header = False);


In [5]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path



In [6]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_loc), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-08-22 16:58:30,348 Reading data from /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/CoMAGC
2019-08-22 16:58:30,350 Train: /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/CoMAGC/train.csv
2019-08-22 16:58:30,351 Dev: /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/CoMAGC/dev.csv
2019-08-22 16:58:30,352 Test: /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/CoMAGC/test.csv


/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.
/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/flair/data_fetcher.py:463: DeprecationWarnin

In [7]:
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]



In [8]:
# document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   rnn_type = "LSTM",
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

In [9]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)



2019-08-22 16:59:00,903 Computing label dictionary. Progress:


100%|██████████| 656/656 [00:00<00:00, 28886.15it/s]

2019-08-22 16:59:00,931 [b'Positive', b'Negative', b'Gene']


In [10]:
trainer = ModelTrainer(classifier, corpus)


In [11]:
# 7. start the training
# trainer.train(data_loc, max_epochs=10)
trainer.train(data_loc,
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=100)

2019-08-22 16:59:11,753 ----------------------------------------------------------------------------------------------------
2019-08-22 16:59:11,754 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): LSTM(256, 512)
 

2019-08-22 17:00:27,159 epoch 5 - iter 6/21 - loss 0.62741252 - samples/sec: 88.60
2019-08-22 17:00:28,123 epoch 5 - iter 8/21 - loss 0.62809483 - samples/sec: 66.88
2019-08-22 17:00:28,975 epoch 5 - iter 10/21 - loss 0.63815630 - samples/sec: 75.81
2019-08-22 17:00:29,771 epoch 5 - iter 12/21 - loss 0.64563434 - samples/sec: 81.22
2019-08-22 17:00:30,235 epoch 5 - iter 14/21 - loss 0.63401258 - samples/sec: 140.69
2019-08-22 17:00:31,083 epoch 5 - iter 16/21 - loss 0.64054539 - samples/sec: 76.16
2019-08-22 17:00:31,441 epoch 5 - iter 18/21 - loss 0.64428606 - samples/sec: 183.60
2019-08-22 17:00:31,699 epoch 5 - iter 20/21 - loss 0.65290687 - samples/sec: 254.22
2019-08-22 17:00:31,709 ----------------------------------------------------------------------------------------------------
2019-08-22 17:00:31,709 EPOCH 5 done: loss 0.6529 - lr 0.1000
2019-08-22 17:00:31,832 DEV : loss 0.658797562122345 - score 0.6747
2019-08-22 17:00:31,843 BAD EPOCHS (no improvement): 3
2019-08-22 17:00:

2019-08-22 17:01:21,783 epoch 11 - iter 6/21 - loss 0.63419062 - samples/sec: 86.78
2019-08-22 17:01:22,134 epoch 11 - iter 8/21 - loss 0.62771787 - samples/sec: 186.24
2019-08-22 17:01:22,930 epoch 11 - iter 10/21 - loss 0.61650043 - samples/sec: 81.51
2019-08-22 17:01:23,667 epoch 11 - iter 12/21 - loss 0.61137259 - samples/sec: 87.60
2019-08-22 17:01:24,017 epoch 11 - iter 14/21 - loss 0.61274506 - samples/sec: 186.83
2019-08-22 17:01:24,480 epoch 11 - iter 16/21 - loss 0.62343701 - samples/sec: 139.99
2019-08-22 17:01:24,980 epoch 11 - iter 18/21 - loss 0.62409192 - samples/sec: 129.61
2019-08-22 17:01:25,296 epoch 11 - iter 20/21 - loss 0.62687143 - samples/sec: 209.01
2019-08-22 17:01:25,308 ----------------------------------------------------------------------------------------------------
2019-08-22 17:01:25,309 EPOCH 11 done: loss 0.6269 - lr 0.0500
2019-08-22 17:01:25,429 DEV : loss 0.6576548218727112 - score 0.6747
2019-08-22 17:01:25,439 BAD EPOCHS (no improvement): 3
2019-

2019-08-22 17:02:13,570 epoch 17 - iter 4/21 - loss 0.63031145 - samples/sec: 143.83
2019-08-22 17:02:14,425 epoch 17 - iter 6/21 - loss 0.63173402 - samples/sec: 75.40
2019-08-22 17:02:15,271 epoch 17 - iter 8/21 - loss 0.61990737 - samples/sec: 76.26
2019-08-22 17:02:15,630 epoch 17 - iter 10/21 - loss 0.60605505 - samples/sec: 182.14
2019-08-22 17:02:16,379 epoch 17 - iter 12/21 - loss 0.59680831 - samples/sec: 86.41
2019-08-22 17:02:16,920 epoch 17 - iter 14/21 - loss 0.60909086 - samples/sec: 119.82
2019-08-22 17:02:17,628 epoch 17 - iter 16/21 - loss 0.61097035 - samples/sec: 91.32
2019-08-22 17:02:17,929 epoch 17 - iter 18/21 - loss 0.62003040 - samples/sec: 217.74
2019-08-22 17:02:18,377 epoch 17 - iter 20/21 - loss 0.61400080 - samples/sec: 145.46
2019-08-22 17:02:18,387 ----------------------------------------------------------------------------------------------------
2019-08-22 17:02:18,388 EPOCH 17 done: loss 0.6140 - lr 0.0250
2019-08-22 17:02:18,505 DEV : loss 0.66051828

2019-08-22 17:03:00,688 epoch 23 - iter 4/21 - loss 0.62139622 - samples/sec: 131.78
2019-08-22 17:03:01,282 epoch 23 - iter 6/21 - loss 0.60684288 - samples/sec: 109.01
2019-08-22 17:03:01,998 epoch 23 - iter 8/21 - loss 0.61958714 - samples/sec: 90.35
2019-08-22 17:03:02,719 epoch 23 - iter 10/21 - loss 0.61539173 - samples/sec: 89.57
2019-08-22 17:03:03,701 epoch 23 - iter 12/21 - loss 0.60224073 - samples/sec: 65.58
2019-08-22 17:03:04,391 epoch 23 - iter 14/21 - loss 0.60991211 - samples/sec: 94.09
2019-08-22 17:03:05,086 epoch 23 - iter 16/21 - loss 0.60993790 - samples/sec: 93.37
2019-08-22 17:03:05,395 epoch 23 - iter 18/21 - loss 0.61022431 - samples/sec: 211.85
2019-08-22 17:03:05,683 epoch 23 - iter 20/21 - loss 0.60390752 - samples/sec: 227.69
2019-08-22 17:03:05,694 ----------------------------------------------------------------------------------------------------
2019-08-22 17:03:05,695 EPOCH 23 done: loss 0.6039 - lr 0.0250
2019-08-22 17:03:05,804 DEV : loss 0.669255733

...

2019-08-22 17:05:38,258 epoch 47 - iter 0/21 - loss 0.62098527 - samples/sec: 222.76
2019-08-22 17:05:38,677 epoch 47 - iter 2/21 - loss 0.64555730 - samples/sec: 155.07
2019-08-22 17:05:39,427 epoch 47 - iter 4/21 - loss 0.61181247 - samples/sec: 86.31
2019-08-22 17:05:40,519 epoch 47 - iter 6/21 - loss 0.62585476 - samples/sec: 59.04
2019-08-22 17:05:40,938 epoch 47 - iter 8/21 - loss 0.62307743 - samples/sec: 156.81
2019-08-22 17:05:41,914 epoch 47 - iter 10/21 - loss 0.61941812 - samples/sec: 66.11
2019-08-22 17:05:42,618 epoch 47 - iter 12/21 - loss 0.61792195 - samples/sec: 91.95
2019-08-22 17:05:43,012 epoch 47 - iter 14/21 - loss 0.61216163 - samples/sec: 164.91
2019-08-22 17:05:43,869 epoch 47 - iter 16/21 - loss 0.60551925 - samples/sec: 75.54
2019-08-22 17:05:44,234 epoch 47 - iter 18/21 - loss 0.60502398 - samples/sec: 179.10
2019-08-22 17:05:44,576 epoch 47 - iter 20/21 - loss 0.60371824 - samples/sec: 192.84
2019-08-22 17:05:44,585 ----------------------------------------

2019-08-22 17:06:17,497 epoch 53 - iter 0/21 - loss 0.72813374 - samples/sec: 463.00
2019-08-22 17:06:18,001 epoch 53 - iter 2/21 - loss 0.62347253 - samples/sec: 129.97
2019-08-22 17:06:18,704 epoch 53 - iter 4/21 - loss 0.58709100 - samples/sec: 91.92
2019-08-22 17:06:19,274 epoch 53 - iter 6/21 - loss 0.59271001 - samples/sec: 114.00
2019-08-22 17:06:19,785 epoch 53 - iter 8/21 - loss 0.59979059 - samples/sec: 127.60
2019-08-22 17:06:20,450 epoch 53 - iter 10/21 - loss 0.59434853 - samples/sec: 98.19
2019-08-22 17:06:21,208 epoch 53 - iter 12/21 - loss 0.58294495 - samples/sec: 85.46
2019-08-22 17:06:21,820 epoch 53 - iter 14/21 - loss 0.58843324 - samples/sec: 105.93
2019-08-22 17:06:22,718 epoch 53 - iter 16/21 - loss 0.58719884 - samples/sec: 71.81
2019-08-22 17:06:23,425 epoch 53 - iter 18/21 - loss 0.59496262 - samples/sec: 91.79
2019-08-22 17:06:23,763 epoch 53 - iter 20/21 - loss 0.61168746 - samples/sec: 193.55
2019-08-22 17:06:23,768 ----------------------------------------

2019-08-22 17:07:00,773 epoch 59 - iter 0/21 - loss 0.69902205 - samples/sec: 107.22
2019-08-22 17:07:01,493 epoch 59 - iter 2/21 - loss 0.64683841 - samples/sec: 90.28
2019-08-22 17:07:02,325 epoch 59 - iter 4/21 - loss 0.62176538 - samples/sec: 77.61
2019-08-22 17:07:02,851 epoch 59 - iter 6/21 - loss 0.62330638 - samples/sec: 123.48
2019-08-22 17:07:03,174 epoch 59 - iter 8/21 - loss 0.60675701 - samples/sec: 206.25
2019-08-22 17:07:03,582 epoch 59 - iter 10/21 - loss 0.60014945 - samples/sec: 159.43
2019-08-22 17:07:04,405 epoch 59 - iter 12/21 - loss 0.59832045 - samples/sec: 78.72
2019-08-22 17:07:04,789 epoch 59 - iter 14/21 - loss 0.61267122 - samples/sec: 171.43
2019-08-22 17:07:05,598 epoch 59 - iter 16/21 - loss 0.61334305 - samples/sec: 80.14
2019-08-22 17:07:06,450 epoch 59 - iter 18/21 - loss 0.61056064 - samples/sec: 75.77
2019-08-22 17:07:06,840 epoch 59 - iter 20/21 - loss 0.60901406 - samples/sec: 168.21
2019-08-22 17:07:06,846 ----------------------------------------

2019-08-22 17:07:53,790 epoch 65 - iter 0/21 - loss 0.48029828 - samples/sec: 311.69
2019-08-22 17:07:54,222 epoch 65 - iter 2/21 - loss 0.54681720 - samples/sec: 150.51
2019-08-22 17:07:54,632 epoch 65 - iter 4/21 - loss 0.52627038 - samples/sec: 160.89
2019-08-22 17:07:55,357 epoch 65 - iter 6/21 - loss 0.56161145 - samples/sec: 89.41
2019-08-22 17:07:56,387 epoch 65 - iter 8/21 - loss 0.57755713 - samples/sec: 62.61
2019-08-22 17:07:56,695 epoch 65 - iter 10/21 - loss 0.58197194 - samples/sec: 214.38
2019-08-22 17:07:57,150 epoch 65 - iter 12/21 - loss 0.57771589 - samples/sec: 142.58
2019-08-22 17:07:57,860 epoch 65 - iter 14/21 - loss 0.58496420 - samples/sec: 91.24
2019-08-22 17:07:58,220 epoch 65 - iter 16/21 - loss 0.59711290 - samples/sec: 181.77
2019-08-22 17:07:59,270 epoch 65 - iter 18/21 - loss 0.59678689 - samples/sec: 61.41
2019-08-22 17:07:59,614 epoch 65 - iter 20/21 - loss 0.59248528 - samples/sec: 189.63
2019-08-22 17:07:59,625 ---------------------------------------

{'test_score': 0.7195,
 'dev_score_history': [0.6386,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6867,
  0.6747,
  0.6747,
  0.6747,
  0.6867,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6867,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6867,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6747,
  0.6867,
  0.6747,
  0.6867,
  0.6747,
  0.6867,
  0.6867,
  0.6867,
  0.6867,
  0.6867,
  0.6867,
  0.6867,
  0.6867],
 'train_loss_history': [0.7370716333389282,
  0.6475467951524825,
  0.6392097274462382,
  0.6493545685495649,
  0.6529068748156229,
  0.6364194339229947,
  0.631101743096397,
  0.6289642708642142,
  0.6205015778541565,
  0.6212627830959502,
  0.6268714325768607,
  0.621688133194333,
  0.6238366

In [ ]:
import torch
torch.cuda.is_available()

In [12]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load(data_loc+'best-model.pt')


2019-08-22 17:13:35,804 loading file /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/CoMAGC/best-model.pt


In [13]:
sentence = Sentence('The FCRL3 polymorphisms associated with rheumatoid arthritis in a Japanese population are not associated per se with rheumatoid arthritis in a Spanish population.')
classifier.predict(sentence)
print(sentence.labels)

[Positive (0.6524571776390076)]


In [19]:
from tqdm import tqdm

In [21]:
ground_truth = []
predicted_label = []
for index, rows in tqdm(df.iterrows(), total=df.shape[0]):
    if rows['label'] == 'NA':
       ground_truth.append('Negative')
    else:
       ground_truth.append('Positive')
    sentence = Sentence(rows['text'])
    classifier.predict(sentence)
    predicted_label.append(sentence.labels[0].value)
    
    

100%|██████████| 3360/3360 [04:11<00:00, 13.36it/s]


In [24]:
from sklearn.metrics import f1_score, accuracy_score

f1_score(ground_truth, predicted_label, average='micro') 

0.6586309523809524